<a href="https://colab.research.google.com/github/Satwikram/Sign-Language-Recognition/blob/dev/Sign%20Language%20Gesture%20Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Connecting to Kaggle

In [1]:
from google.colab import files

files.upload()


! mkdir ~/.kaggle


! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


### Downloading dataset

In [2]:
!kaggle datasets download -d grassknoted/asl-alphabet

 99% 1.02G/1.03G [00:17<00:00, 88.7MB/s]
100% 1.03G/1.03G [00:17<00:00, 64.0MB/s]


### Unzipping Dataset

In [ ]:
!unzip /content/asl-alphabet.zip

### Importing Dependencies

In [5]:
import numpy as np
import pandas as pd
import os
import shutil
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from pathlib import Path

### Loading Dataset

In [9]:
shutil.rmtree("/content/asl_alphabet_train/asl_alphabet_train/del")

In [10]:
shutil.rmtree("/content/asl_alphabet_train/asl_alphabet_train/nothing")

In [11]:
shutil.rmtree("/content/asl_alphabet_train/asl_alphabet_train/space")

In [11]:
uniq_labels = sorted(os.listdir('/content/asl_alphabet_train/asl_alphabet_train'))

In [12]:
uniq_labels

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z']

In [13]:
len(uniq_labels)

26

### Function to load dataset

In [14]:
def imgPaths(filepath):

    labels = [str(filepath[i]).split("/")[-2] \
              for i in range(len(filepath))]

    filepath = pd.Series(filepath, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    # Concatenate filepaths and labels
    df = pd.concat([filepath, labels], axis=1)

    # Shuffle the DataFrame and reset index
    df = df.sample(frac=1).reset_index(drop = True)
    
    return df

In [15]:
def imgPaths1(filepath):

    labels = [str(filepath[i]).split("/")[-1] \
              for i in range(len(filepath))]

    filepath = pd.Series(filepath, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    # Concatenate filepaths and labels
    df = pd.concat([filepath, labels], axis=1)

    # Shuffle the DataFrame and reset index
    df = df.sample(frac=1).reset_index(drop = True)
    
    return df

In [16]:
train_image_dir = Path('/content/asl_alphabet_train/asl_alphabet_train')
train_filepaths = list(train_image_dir.glob(r'**/*.jpg'))

test_image_dir = Path('/content/asl_alphabet_test/asl_alphabet_test')
test_filepaths = list(test_image_dir.glob(r'**/*.jpg'))

In [17]:
train_df = imgPaths(train_filepaths)
test_df = imgPaths1(test_filepaths)

In [18]:
p = test_df["Label"].shape[0]

In [19]:
a=[]

for i in range(p):
    test_df["Label"][i]=test_df["Label"][i][:-9]

In [20]:
test_df

,Filepath,Label
0,/content/asl_alphabet_test/asl_alphabet_test/G...,G
1,/content/asl_alphabet_test/asl_alphabet_test/Y...,Y
2,/content/asl_alphabet_test/asl_alphabet_test/U...,U
3,/content/asl_alphabet_test/asl_alphabet_test/L...,L
4,/content/asl_alphabet_test/asl_alphabet_test/P...,P
5,/content/asl_alphabet_test/asl_alphabet_test/O...,O
6,/content/asl_alphabet_test/asl_alphabet_test/K...,K
7,/content/asl_alphabet_test/asl_alphabet_test/C...,C
8,/content/asl_alphabet_test/asl_alphabet_test/I...,I
9,/content/asl_alphabet_test/asl_alphabet_test/S...,S


In [21]:
df_unique = train_df.copy().drop_duplicates(subset=["Label"]).reset_index()

In [22]:
df_unique

,index,Filepath,Label
0,0,/content/asl_alphabet_train/asl_alphabet_train...,S
1,1,/content/asl_alphabet_train/asl_alphabet_train...,W
2,2,/content/asl_alphabet_train/asl_alphabet_train...,H
3,3,/content/asl_alphabet_train/asl_alphabet_train...,T
4,4,/content/asl_alphabet_train/asl_alphabet_train...,R
5,5,/content/asl_alphabet_train/asl_alphabet_train...,V
6,6,/content/asl_alphabet_train/asl_alphabet_train...,Y
7,7,/content/asl_alphabet_train/asl_alphabet_train...,L
8,9,/content/asl_alphabet_train/asl_alphabet_train...,I
9,10,/content/asl_alphabet_train/asl_alphabet_train...,X


In [26]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [27]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(300, 300),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

Found 62400 validated image filenames belonging to 26 classes.
Found 15600 validated image filenames belonging to 26 classes.
Found 26 validated image filenames belonging to 26 classes.


In [29]:
pretrainedModel = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
     include_top=False,
     weights='imagenet',
     pooling='avg'
)
pretrainedModel.trainable = False

In [30]:
inputs = pretrainedModel.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrainedModel.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(26, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

adam = tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam'
)

model.compile(
    optimizer=adam,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [31]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/Sign Language Prediction/Gesture/model.h5',
                              monitor = 'val_loss',
                              save_best_only = True,
                              verbose = 1,
                              mode = 'min')

In [32]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor = 0.2,
                              patience = 2, min_lr = 0.001, verbose = 1)

In [33]:
def scheduler(epoch, lr):
  if epoch < 2:
    return lr
  else:
    return lr * tf.math.exp(-0.1)

In [34]:
lr_schedule = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose = 1)

In [38]:
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta = 0, 
                                             patience = 2, verbose = 1, mode='min',
                                             restore_best_weights = True)

In [37]:
callbacks = [checkpoint, reduce_lr, lr_schedule, earlystop]

In [ ]:
history = model.fit(
    train_images,
    validation_data=val_images,
    epochs = 5,
    callbacks = callbacks)
     

Epoch 1/5

Epoch 00001: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
 301/1950 [===>..........................] - ETA: 26:18 - loss: 0.3183 - accuracy: 0.9065